In [1]:
pip install urlexpander


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import urlexpander

In [3]:
df = pd.read_csv("../../df_tweets_4_months_urls.csv") 

In [4]:
df_data_2_one = df[df.n_urls == 1]

In [5]:
df_data_2_one.urls[0][2:-2]

'https://t.co/5djnLz36zv'

In [6]:
df_data_2_one["short_url"] =  df_data_2_one.urls.apply(lambda x: x[2:-2])


<ipython-input-6-92c066dd6cc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_2_one["short_url"] =  df_data_2_one.urls.apply(lambda x: x[2:-2])


In [7]:
def unshorten_one(url):
    try:
        short =  urlexpander.expand(url)

    except requests.ConnectionError:
            short = "connection error"
    except:
        short = urlexpander.expand("http://" + url)
    try:
    
        if "__CLIENT_ERROR__"  in urlexpander.expand(short):     
            try:
                r = requests.get(url, allow_redirects=False)
            except requests.MissingSchema:
                r = requests.get("http://" + url, allow_redirects=False)
            except requests.ConnectionError:
                short = "connection error"

            try:
                short = r.headers['location']
            except KeyError:
                short =  "Page doesn't exist!"
    except requests.ConnectionError:
        short = "connection error"
       
    return short

In [8]:
def unshorten_one(url):
    try:
        short = urlexpander.expand(url)

    except requests.ConnectionError:
        short = "connection error"
    except:
        short = urlexpander.expand("http://" + url)
    try:

        if "__CLIENT_ERROR__" in urlexpander.expand(short):
            try:
                r = requests.get(url, allow_redirects=False)
                try:
                    short = r.headers["location"]
                except KeyError:
                    short = "Page doesn't exist!"
            except requests.exceptions.MissingSchema:
                r = requests.get("http://" + url, allow_redirects=False)
                try:
                    short = r.headers["location"]
                except KeyError:
                    short = "Page doesn't exist!"
            except requests.ConnectionError:
                short = "connection error"


    except requests.ConnectionError:
        short = "connection error"

    return short

In [ ]:
import time
start_time = time.time()
df_test = df_data_2_one.short_url.iloc[20000:21000].apply(lambda x: unshorten_one(x))

In [ ]:
df_test